<a href="https://colab.research.google.com/github/ksheng-/nyc-housing/blob/master/colab/altair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependencies and modules

In [0]:
!pip install --upgrade geopandas

In [0]:
import requests
import psycopg2
import psycopg2.extras
import pprint
import altair as alt
import pandas as pd
import numpy as np
from vega_datasets import data

# Retrieve and clean data

## Gross income

In [0]:
gross_income = pd.read_csv('http://taxbills.nyc/gross_income.csv', dtype={'bbl': str})

In [0]:
# manhattan only
gross_income_ = gross_income[gross_income['bbl'].str[0] == '1']

# drop columns without coordinates
gross_income_ = gross_income_.dropna(subset=['lon', 'lat', 'unitstotal'])
gross_income_ = gross_income_[gross_income_['unitstotal'] != 0]

# normalize yearly income by # units
income_cols = ['income' + str(i).zfill(2) for i in range(7, 15)]
normalized_cols = [s + '_normalized' for s in income_cols]
gross_income_[normalized_cols] = gross_income_[income_cols].div(gross_income_['unitstotal'], axis=0)
gross_income_ = gross_income_.sample(5000)
gross_income_.hist(column=normalized_cols, bins=100)
print(np.nanmean(gross_income_[normalized_cols]))
print(np.nanstd(gross_income_[normalized_cols]))
gross_income_[normalized_cols] = (gross_income_[normalized_cols] - np.nanmean(gross_income_[normalized_cols]) / np.nanstd(gross_income_[normalized_cols])).clip(-2, 2)


## Rent stabilization

In [0]:
conn_params = {
    'host': '157.230.184.188',
    'dbname': 'nyc_db',
    'user': 'postgres',
    'password': 'password'
}
conn = psycopg2.connect(**conn_params)

query = '''
    WITH hpd AS (
        SELECT bbl, count(*) 
        AS nviolations
        FROM hpd_violations
        GROUP BY bbl
    )
    SELECT rentstab.ucbbl, address, ownername, lat, lon, nviolations, corpnames, ownernames,
        unitsres, unitstotal, uc2007, uc2008, uc2009, uc2010, uc2011, uc2012, uc2013, uc2014, uc2014 - uc2007 as diff, (uc2014 - uc2007) / unitstotal::decimal as pct
    FROM rentstab
    INNER JOIN hpd
      ON rentstab.ucbbl = hpd.bbl
    INNER JOIN hpd_registrations_grouped_by_bbl_with_contacts AS registrations 
      ON hpd.bbl = registrations.bbl
    WHERE borough = 'MN'
      AND unitstotal > 0
      AND uc2014 IS NOT NULL
'''

rent_stab = pd.read_sql(query, conn)

## Geodata

In [0]:
# use geopandas to manipulate geojson files (altair datasets are topojson but geojson is more common): https://github.com/altair-viz/altair/issues/588
import geopandas as gpd

# bureau boundaries: https://data.cityofnewyork.us/City-Government/Borough-Boundaries/tqmj-j8zm
data = requests.get('https://data.cityofnewyork.us/api/geospatial/tqmj-j8zm?method=export&format=GeoJSON').json()
bureau_boundaries_gdf = gpd.GeoDataFrame.from_features(data)
bureau_boundaries_gdf = bureau_boundaries_gdf[bureau_boundaries_gdf['boro_name'] == "Manhattan"]
bureau_boundaries = alt.InlineData(values=bureau_boundaries_gdf.to_json(), format=alt.DataFormat(property='features',type='json'))

# subway lines: https://data.cityofnewyork.us/Transportation/Subway-Lines/3qz8-muuu
data = requests.get('https://data.cityofnewyork.us/api/geospatial/3qz8-muuu?method=export&format=GeoJSON').json()

subway_lines_gdf = gpd.GeoDataFrame.from_features(data)

xmin, ymin, xmax, ymax = bureau_boundaries_gdf.total_bounds
subway_lines_gdf = subway_lines_gdf.cx[xmin:xmax, ymin:ymax]

subway_lines = alt.InlineData(values=subway_lines_gdf.to_json(), format=alt.DataFormat(property='features', type='json'))

# Render charts

In [0]:
c0 = alt.Chart(gross_income_).mark_circle(size=8).encode(
    alt.Color(alt.repeat('column'), type='quantitative'),
    longitude='lon:Q',
    latitude='lat:Q'
)
c1 = alt.Chart(bureau_boundaries).mark_geoshape(fillOpacity=0, color='gray', stroke='gray').encode(
    tooltip='properties.boro_name:N'
)

c2 = alt.Chart(subway_lines).mark_geoshape(opacity=.1, fillOpacity=0, stroke='red').encode(
    tooltip='properties.name:N'
)

(c0 + c1 + c2).properties(width=200, height=400).repeat(
    column=[normalized_cols[i] for i in [0, 2, 5, 7]],
)

In [0]:
alt.Chart(gross_income3).mark_circle(size=8).encode(
    longitude='lon:Q',
    latitude='lat:Q',
    color='income09:Q'
).properties(
    width=200,
    height=500
)


In [0]:
alt.Chart(gross_income3).mark_circle(size=8).encode(
    longitude='lon:Q',
    latitude='lat:Q',
    color='income14:Q'
).properties(
    width=200,
    height=500
)
